In [135]:
from tensorflow import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf
import xgboost as xgb
import joblib

In [136]:
original_dim = 27
intermediate_dim = 12
latent_dim = 5

inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)


In [137]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_sigma = inputs
        
        batch = tf.shape(z_mean)[0]
        
        dim = tf.shape(z_mean)[1]
        
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_sigma) * epsilon

z = Sampling()([z_mean, z_log_sigma])

In [139]:
# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='tanh')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

In [140]:
mse = keras.losses.MeanSquaredError()
reconstruction_loss = mse(inputs, outputs)
reconstruction_loss *= 27

kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

vae_loss = K.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
optimizer=keras.optimizers.Adam()
vae.compile(optimizer=optimizer)

In [141]:
dataset = pd.read_csv('datasets/Fs_B_DANCE_WALK_0.5sec.csv')
dataset = dataset.drop(columns=['EMOTION_P', 'EMOTION_A', 'EMOTION_D'])

train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

train_dataset = np.asarray(train_dataset)
test_dataset = np.asarray(test_dataset)

x_train = train_dataset.reshape((len(train_dataset), np.prod(train_dataset.shape[1:])))
x_test = test_dataset.reshape((len(test_dataset), np.prod(test_dataset.shape[1:])))

print(len(x_train[0]))

No Training Samples: 14734
No Test Samples: 3684
27


In [143]:
vae.fit(x_train, x_train,
        epochs=64,
        batch_size=16,
       )

Epoch 1/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6187
Epoch 2/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6232
Epoch 3/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6028
Epoch 4/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6202
Epoch 5/64
921/921 [==============================] - 1s 1ms/step - loss: 18.5948
Epoch 6/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6000
Epoch 7/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6059
Epoch 8/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6126
Epoch 9/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6004
Epoch 10/64
921/921 [==============================] - 1s 1ms/step - loss: 18.5978
Epoch 11/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6027
Epoch 12/64
921/921 [==============================] - 1s 1ms/step - loss: 18.6132
Epoch 13/64
9

In [144]:
sample = np.asarray(x_test[1])
sample = sample.reshape(1,-1)
print(sample)

[[ 0.29577057  0.1995598   0.16912828  0.52961919  0.4191004   0.42236775
   0.29504198  0.29957959  0.286151    0.2699125  -0.19302982 -0.25811558
  -0.07891554  0.93960203  0.19899026  0.0865777   0.19354236  0.77026704
   0.82223654  0.38248285  1.10377377  0.21948754  1.22960612  1.64132263
   1.25433136  2.40144299  0.29973441]]


In [145]:
mean, var, generated = encoder.predict(sample)
print(generated)

[[-9.4489902e-01 -1.7580755e-02 -1.8120952e-01  7.4662268e-05
   9.0170878e-01]]


In [146]:
regen = decoder.predict(generated)
print(regen)

[[ 0.660388    0.38778755  0.37952667  0.52129537  0.4409387   0.43442106
   0.34387523  0.34041178  0.2862039   0.27434883 -0.03574353  0.11459128
  -0.05878445  0.720847    0.42815375  0.10664046  0.17576106  0.82648754
   0.80613637  0.5783576   0.6343103   0.47255474  1.          1.
   1.          1.          0.9982845 ]]


In [133]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [147]:
scaled_sample = scaler.transform(sample)

real_coordinates = (
    model_p.predict(scaled_sample),
    model_a.predict(scaled_sample),
    model_d.predict(scaled_sample)
)

scaled_regen = scaler.transform(regen)

generated_coordinates = (
    model_p.predict(scaled_regen),
    model_a.predict(scaled_regen),
    model_d.predict(scaled_regen)
)


print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % np.asarray(generated_coordinates))

Real: [[ 0.11446296]
 [-0.12988538]
 [-0.11219636]]
Predicted: [[0.21994561]
 [0.05928715]
 [0.2291142 ]]


/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
